In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_allopenclosed AS
SELECT DISTINCT
    rd.visualrequestfilenumber,
    rd.groupname,
    rd.caseowner,
    rd.ministry,
    CASE
        WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
        ELSE rd.officeid
    END AS officecode,
    rt.requesttypename,
    category.requestertypename,
    req.fullname,
    rd.subject,
    rd.startdate,
    rd.targetdate,
    CASE
        WHEN rd.requeststatus = 'Closed' THEN rd.closeddate
        ELSE NULL
    END AS closeddate,
    rd.processeddays,
    rd.currentactivity,
    rd.primaryusername,
    rd.crossgovtno,
    rd.requestdescription,
    rs.requeststatusname,
    custom.countontime,
    custom.countoverdue,
    custom.onholddays,
    custom.daysoverdue,
    pt.estimatedamount,
    pt.feewaived,
    custom.feepaidamount,
    rd.disposition,
    rd.publication,
    rd.publicationreason,
    ext.numberofextensions,
    rdd.noofpagesintherequest,
    frd.recordspagecount
FROM
    factRequestDetails AS rd
    LEFT JOIN dimRequests AS r ON (rd.foirequestid = r.foirequestid AND rd.sourceoftruth = r.sourceoftruth)
    AND r.cactive = 'Y'
    LEFT JOIN dimRequestTypes AS rt ON rt.requesttypeid = rd.requesttypeid
    AND rt.cactive = 'Y'
    LEFT JOIN factRequestDocumentsDetails AS rdd ON rdd.foirequestid = rd.foirequestid
    AND rdd.activeflag = 'Y'
    LEFT JOIN
        dimECOffice AS eco ON CASE
            WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officeid = rd.officeid
            ELSE FALSE
        END
    LEFT JOIN dimRequestStatuses AS rs ON rd.requeststatus = rs.requeststatus
    LEFT JOIN factRequestRequesters AS rr ON rd.foirequestid = rr.foirequestid
    AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes AS category ON rd.applicantcategoryid = category.requestertypeid
    LEFT JOIN dimRequesters AS req ON (
        (req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD')
        AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD')
        AND rd.requesterid = req.requesterid
    ) OR (
        req.sourceoftruth = 'FOIMOD'
        AND rd.sourceoftruth = 'FOIMOD'
        AND req.foirequestid = rd.foirequestid
        AND req.requesterid = rd.requesterid
    )
    LEFT JOIN factRequestCustomCalcFields AS custom ON rd.foirequestid = custom.foirequestid
    LEFT JOIN factRequestInvoices AS ri ON rd.foirequestid = ri.foirequestid
    AND ri.activeflag = 'Y'
    LEFT JOIN dimMODMinistryRequestIDs AS rid ON rd.foirequestid = rid.foirequestid
    LEFT JOIN (
        SELECT
            foirequestid,
            COUNT(DISTINCT requestextid) AS numberofextensions
        FROM
            factRequestExtensions
        WHERE
            approvedstatus = 'A'
            AND activeflag = 'Y'
        GROUP BY
            foirequestid
    ) AS ext ON rd.foirequestid = ext.foirequestid
    LEFT JOIN (
        SELECT DISTINCT
            a.foirequestnumber,
            a.recordspagecount
        FROM
            factFOIFlowRequestStatusDetails AS a
            JOIN (
                SELECT
                    foirequestnumber,
                    MAX(version) AS maxv
                FROM
                    factFOIFlowRequestStatusDetails
                WHERE
                    recordspagecount IS NOT NULL
                    AND isactive = TRUE
                GROUP BY
                    foirequestnumber
            ) AS c ON c.foirequestnumber = a.foirequestnumber
            AND c.maxv = a.version
        WHERE
            a.recordspagecount IS NOT NULL
            AND a.isactive = TRUE
    ) AS frd ON frd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestPaymentTransaction AS pt ON rd.foirequestid = pt.foirequestid
    AND pt.activeflag = 'Y'
    JOIN (
        SELECT
            MAX(runcycleid) AS maxruncycleid,
            visualrequestfilenumber
        FROM
            factRequestDetails
        WHERE
            activeflag = 'Y'
        GROUP BY
            visualrequestfilenumber
    ) AS maxruncycle ON rd.visualrequestfilenumber = maxruncycle.visualrequestfilenumber
    AND rd.runcycleid = maxruncycle.maxruncycleid
WHERE
    rd.activeflag = 'Y'
    AND (
        rd.closeddate >= TIMESTAMP('2019-04-01 00:00:00')
        OR rd.closeddate = TIMESTAMP('1900-01-01 00:00:00')
    )
    AND eco.officecode NOT IN ('IAO', 'IMB', 'XGR', 'PROAC');


In [0]:
%sql

--optimize factRequestDivisionalStages after ETL
OPTIMIZE factRequestDivisionalStages
ZORDER BY (foirequestnumber);

CREATE OR REPLACE TABLE default.materialized_viw_analystworkloaddash AS
SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(try_cast(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(try_cast(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid and ecg.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname and ecggn.sourceoftruth = rd.sourceoftruth
    LEFT JOIN
        dimECOffice AS eco ON CASE
            WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officeid = rd.officeid
            ELSE FALSE
        END
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid and rd.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid and rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid and s.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid and rd.sourceoftruth = d.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN ("1", "10", "21", "33", "41", "45", "49", "51", "55", 'CLB', 'IIO', 'MGC', 'OBC', 'TIC')
  AND rd.requesttypeid IN (31, 33);
